# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load test data

In [2]:
# dataDir = "data/2D_Truss_v1.3/conmech/"
# testDir = os.path.join(dataDir, 'design_9_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

# dataDir = "data/endLoadsv1.0/conmech/"
# testDir = os.path.join(dataDir, 'design_7_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

dataDir = "data/tower1.0/conmech/"
testDir = os.path.join(dataDir, 'design_5_N_1000/')
allGraphsUnfiltered = loadConmechGraphs(testDir, loadDims=[0])

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.027434
std,0.005930
min,0.022934
25%,0.023780
50%,0.025250
75%,0.028404
max,0.065064


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.025836
std,0.002804
min,0.022934
25%,0.023687
50%,0.024775
75%,0.027176
max,0.034477


## 2. Load pre-train data

In [4]:
# pretrainDirs = glob.glob(os.path.join(dataDir, '*1000/'))
# pretrainDirs.remove(testDir)

pretrainDirs = ['data/2D_Truss_v1.3/conmech/design_7_N_1000/']

allPretrainGraphs = []
for pretrainDir in pretrainDirs:
    designName = pretrainDir.split('/')[-2].split('_N')[0]
    print(f'loading from {pretrainDir}')
    graphsUnfiltered = loadConmechGraphs(pretrainDir)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading from data/2D_Truss_v1.3/conmech/design_7_N_1000/
loaded 900 pretraining graphs


## 3. Initial training

In [5]:
saveDir = 'results/transferLrn_des7_tower5_03/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

epoch: 0   trainLoss: 9.1679e-01   valLoss:8.3533e-01  time: 2.15e+00
epoch: 1   trainLoss: 7.5296e-01   valLoss:8.4459e-01  time: 1.81e+00
epoch: 2   trainLoss: 6.4657e-01   valLoss:9.2376e-01  time: 1.82e+00
epoch: 3   trainLoss: 5.7322e-01   valLoss:1.1124e+00  time: 1.83e+00
epoch: 4   trainLoss: 5.0579e-01   valLoss:1.3154e+00  time: 1.83e+00
epoch: 5   trainLoss: 4.6400e-01   valLoss:2.0401e+00  time: 1.84e+00
epoch: 6   trainLoss: 4.0377e-01   valLoss:3.0466e+00  time: 1.81e+00
epoch: 7   trainLoss: 3.5834e-01   valLoss:3.6507e+00  time: 1.84e+00
epoch: 8   trainLoss: 3.1649e-01   valLoss:3.2505e+00  time: 1.83e+00
epoch: 9   trainLoss: 2.8219e-01   valLoss:2.2849e+00  time: 1.83e+00
epoch: 10   trainLoss: 2.6543e-01   valLoss:1.4233e+00  time: 1.81e+00
epoch: 11   trainLoss: 2.2674e-01   valLoss:1.0754e+00  time: 1.81e+00
epoch: 12   trainLoss: 2.3209e-01   valLoss:8.7757e-01  time: 1.80e+00
epoch: 13   trainLoss: 2.1179e-01   valLoss:8.9913e-01  time: 1.82e+00
epoch: 14   trai

alt.Chart(...)

In [6]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000008,0.001856,0.097898,0.875786,0.956675,8.136294e-01,0.000000e+00
test,0.572653,0.575528,0.942815,-422969.078081,0.000000,-2.199252e+06,-1.022990e+07


## 4. Transfer learning study

In [7]:
# trainDataDirs = glob.glob(os.path.join(dataDir, 'design_9*/'))
trainDataDirs = glob.glob(os.path.join(dataDir, '*/'))
trainDataDirs.remove(testDir)

allResults = []
for trainDataDir in trainDataDirs:
#     trainDataUnfiltered = loadConmechGraphs(trainDataDir)
    trainDataUnfiltered = loadConmechGraphs(trainDataDir, loadDims=[0])

    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'reading from {trainDataDir}')
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    plotHistory(history)
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    plotHistory(history)
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                  useXFeatures=False,
                  saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

reading from data/tower1.0/conmech/design_5_N_200/
loaded train set of size 180
epoch: 0   trainLoss: 1.0240e+00   valLoss:9.1496e-01  time: 1.56e+00
epoch: 1   trainLoss: 7.8112e-01   valLoss:8.9729e-01  time: 1.53e+00
epoch: 2   trainLoss: 5.8695e-01   valLoss:8.7684e-01  time: 1.51e+00
epoch: 3   trainLoss: 4.2135e-01   valLoss:8.7036e-01  time: 1.52e+00
epoch: 4   trainLoss: 2.9587e-01   valLoss:8.5430e-01  time: 1.49e+00
epoch: 5   trainLoss: 2.1318e-01   valLoss:8.3181e-01  time: 1.51e+00
epoch: 6   trainLoss: 1.4222e-01   valLoss:7.9850e-01  time: 1.51e+00
epoch: 7   trainLoss: 1.0168e-01   valLoss:7.6207e-01  time: 1.51e+00
epoch: 8   trainLoss: 8.3090e-02   valLoss:7.2234e-01  time: 1.55e+00
epoch: 9   trainLoss: 8.1100e-02   valLoss:6.7390e-01  time: 1.49e+00
epoch: 10   trainLoss: 8.3343e-02   valLoss:6.2379e-01  time: 1.49e+00
epoch: 11   trainLoss: 8.2423e-02   valLoss:5.5289e-01  time: 1.50e+00
epoch: 12   trainLoss: 7.6954e-02   valLoss:4.7632e-01  time: 1.50e+00
epoch: 

epoch: 14   trainLoss: 5.2391e-02   valLoss:7.9953e-02  time: 1.54e+00
epoch: 15   trainLoss: 5.0502e-02   valLoss:7.4834e-02  time: 1.54e+00
epoch: 16   trainLoss: 4.8075e-02   valLoss:7.1996e-02  time: 1.57e+00
epoch: 17   trainLoss: 4.5794e-02   valLoss:6.9901e-02  time: 1.56e+00
epoch: 18   trainLoss: 4.3628e-02   valLoss:6.8286e-02  time: 1.55e+00
epoch: 19   trainLoss: 4.1384e-02   valLoss:6.6071e-02  time: 1.59e+00
epoch: 20   trainLoss: 3.9340e-02   valLoss:6.3342e-02  time: 1.55e+00
epoch: 21   trainLoss: 3.7435e-02   valLoss:6.0276e-02  time: 1.56e+00
epoch: 22   trainLoss: 3.6229e-02   valLoss:5.6640e-02  time: 1.58e+00
epoch: 23   trainLoss: 3.5315e-02   valLoss:5.3432e-02  time: 1.55e+00
epoch: 24   trainLoss: 3.3913e-02   valLoss:5.1662e-02  time: 1.55e+00
epoch: 25   trainLoss: 3.2514e-02   valLoss:5.0354e-02  time: 1.55e+00
epoch: 26   trainLoss: 3.1099e-02   valLoss:4.8795e-02  time: 1.54e+00
epoch: 27   trainLoss: 2.9638e-02   valLoss:4.8328e-02  time: 1.55e+00
epoch:

epoch: 28   trainLoss: 1.2478e-02   valLoss:8.2146e-02  time: 3.64e+00
epoch: 29   trainLoss: 1.1252e-02   valLoss:7.8661e-02  time: 3.67e+00
epoch: 30   trainLoss: 1.0426e-02   valLoss:7.7735e-02  time: 3.64e+00
epoch: 31   trainLoss: 9.9164e-03   valLoss:6.9563e-02  time: 3.62e+00
epoch: 32   trainLoss: 8.9558e-03   valLoss:7.1810e-02  time: 3.63e+00
epoch: 33   trainLoss: 9.4057e-03   valLoss:6.3879e-02  time: 3.61e+00
epoch: 34   trainLoss: 7.9179e-03   valLoss:5.5095e-02  time: 3.64e+00
epoch: 35   trainLoss: 6.9451e-03   valLoss:5.7783e-02  time: 3.76e+00
epoch: 36   trainLoss: 5.8971e-03   valLoss:5.2323e-02  time: 3.65e+00
epoch: 37   trainLoss: 5.0184e-03   valLoss:5.4163e-02  time: 3.62e+00
epoch: 38   trainLoss: 3.4968e-03   valLoss:5.0693e-02  time: 3.61e+00
epoch: 39   trainLoss: 3.2444e-03   valLoss:4.2988e-02  time: 3.61e+00
epoch: 40   trainLoss: 3.0880e-03   valLoss:4.5141e-02  time: 3.65e+00
epoch: 41   trainLoss: 2.4810e-03   valLoss:3.4646e-02  time: 3.69e+00
epoch:

epoch: 43   trainLoss: 2.3589e-03   valLoss:4.9308e-03  time: 3.63e+00
epoch: 44   trainLoss: 1.9721e-03   valLoss:8.6349e-03  time: 3.65e+00
epoch: 45   trainLoss: 1.6499e-03   valLoss:1.6424e-02  time: 3.64e+00
epoch: 46   trainLoss: 1.6369e-03   valLoss:1.3295e-02  time: 3.66e+00
epoch: 47   trainLoss: 1.4592e-03   valLoss:8.2018e-03  time: 3.65e+00
epoch: 48   trainLoss: 9.6504e-04   valLoss:5.8632e-03  time: 3.60e+00
epoch: 49   trainLoss: 1.2173e-03   valLoss:2.2219e-03  time: 3.71e+00
epoch: 50   trainLoss: 1.2942e-03   valLoss:7.3752e-03  time: 3.63e+00
epoch: 51   trainLoss: 2.0161e-03   valLoss:2.4313e-03  time: 3.62e+00
epoch: 52   trainLoss: 1.9502e-03   valLoss:3.6176e-03  time: 3.64e+00
epoch: 53   trainLoss: 2.2178e-03   valLoss:3.3448e-03  time: 3.63e+00
epoch: 54   trainLoss: 9.8526e-04   valLoss:3.2538e-03  time: 3.61e+00
epoch: 55   trainLoss: 8.5776e-04   valLoss:7.4027e-03  time: 3.61e+00
epoch: 56   trainLoss: 1.9209e-03   valLoss:3.5708e-03  time: 3.61e+00
epoch:

epoch: 58   trainLoss: 1.8328e-02   valLoss:7.1263e-02  time: 9.80e-02
epoch: 59   trainLoss: 1.7737e-02   valLoss:6.9755e-02  time: 9.15e-02
epoch: 60   trainLoss: 1.7322e-02   valLoss:6.9139e-02  time: 9.65e-02
epoch: 61   trainLoss: 1.6800e-02   valLoss:6.7755e-02  time: 9.30e-02
epoch: 62   trainLoss: 1.6463e-02   valLoss:6.5624e-02  time: 9.34e-02
epoch: 63   trainLoss: 1.5951e-02   valLoss:6.5114e-02  time: 9.10e-02
epoch: 64   trainLoss: 1.5407e-02   valLoss:6.5108e-02  time: 9.32e-02
epoch: 65   trainLoss: 1.4945e-02   valLoss:6.4311e-02  time: 9.14e-02
epoch: 66   trainLoss: 1.4491e-02   valLoss:6.3776e-02  time: 9.13e-02
epoch: 67   trainLoss: 1.4110e-02   valLoss:6.3913e-02  time: 9.13e-02
epoch: 68   trainLoss: 1.3774e-02   valLoss:6.3831e-02  time: 9.23e-02
epoch: 69   trainLoss: 1.3427e-02   valLoss:6.3802e-02  time: 9.16e-02
epoch: 70   trainLoss: 1.3029e-02   valLoss:6.3791e-02  time: 9.14e-02
epoch: 71   trainLoss: 1.2533e-02   valLoss:6.3592e-02  time: 9.10e-02
epoch:

...

epoch: 51   trainLoss: 2.8886e-02   valLoss:6.3328e-02  time: 4.94e-02
epoch: 52   trainLoss: 2.8798e-02   valLoss:6.5859e-02  time: 4.93e-02
epoch: 53   trainLoss: 2.8687e-02   valLoss:6.8317e-02  time: 4.95e-02
epoch: 54   trainLoss: 2.8588e-02   valLoss:7.0921e-02  time: 4.88e-02
epoch: 55   trainLoss: 2.8470e-02   valLoss:7.3852e-02  time: 4.92e-02
epoch: 56   trainLoss: 2.8311e-02   valLoss:7.6663e-02  time: 5.06e-02
epoch: 57   trainLoss: 2.8103e-02   valLoss:7.9071e-02  time: 4.89e-02
epoch: 58   trainLoss: 2.7828e-02   valLoss:8.1022e-02  time: 4.90e-02
epoch: 59   trainLoss: 2.7406e-02   valLoss:8.3302e-02  time: 5.16e-02
epoch: 60   trainLoss: 2.6818e-02   valLoss:8.5961e-02  time: 4.95e-02
epoch: 61   trainLoss: 2.6114e-02   valLoss:8.4080e-02  time: 4.98e-02
epoch: 62   trainLoss: 2.5178e-02   valLoss:7.4569e-02  time: 4.93e-02
epoch: 63   trainLoss: 2.4397e-02   valLoss:7.6472e-02  time: 4.94e-02
epoch: 64   trainLoss: 2.3756e-02   valLoss:7.5360e-02  time: 4.88e-02
epoch:

epoch: 65   trainLoss: 1.1035e-03   valLoss:2.3847e-02  time: 7.32e+00
epoch: 66   trainLoss: 1.6882e-03   valLoss:1.4350e-02  time: 7.29e+00
epoch: 67   trainLoss: 1.7676e-03   valLoss:2.1857e-02  time: 7.30e+00
epoch: 68   trainLoss: 1.8205e-03   valLoss:2.0415e-02  time: 7.22e+00
epoch: 69   trainLoss: 5.0439e-03   valLoss:9.8256e-03  time: 7.31e+00
epoch: 70   trainLoss: 3.1040e-03   valLoss:1.3885e-02  time: 7.36e+00
epoch: 71   trainLoss: 2.0010e-03   valLoss:1.1771e-02  time: 7.30e+00
epoch: 72   trainLoss: 1.8398e-03   valLoss:5.3538e-03  time: 7.05e+00
epoch: 73   trainLoss: 1.8904e-03   valLoss:6.7707e-03  time: 7.13e+00
epoch: 74   trainLoss: 3.3174e-03   valLoss:7.7292e-03  time: 7.11e+00
epoch: 75   trainLoss: 2.2901e-03   valLoss:1.3072e-02  time: 7.06e+00
epoch: 76   trainLoss: 1.8028e-03   valLoss:1.4231e-02  time: 7.18e+00
epoch: 77   trainLoss: 1.6744e-03   valLoss:1.5302e-02  time: 7.23e+00
epoch: 78   trainLoss: 1.5429e-03   valLoss:1.9527e-02  time: 7.17e+00
epoch:

epoch: 80   trainLoss: 1.2033e-03   valLoss:3.5856e-03  time: 7.30e+00
epoch: 81   trainLoss: 2.6827e-03   valLoss:6.4507e-03  time: 7.37e+00
epoch: 82   trainLoss: 2.2796e-03   valLoss:4.1399e-03  time: 7.42e+00
epoch: 83   trainLoss: 1.6256e-03   valLoss:3.8682e-03  time: 7.31e+00
epoch: 84   trainLoss: 1.7521e-03   valLoss:3.8034e-03  time: 7.24e+00
epoch: 85   trainLoss: 1.5351e-03   valLoss:2.7038e-03  time: 7.28e+00
epoch: 86   trainLoss: 1.8329e-03   valLoss:2.5617e-03  time: 7.24e+00
epoch: 87   trainLoss: 2.4852e-03   valLoss:2.9439e-03  time: 7.33e+00
epoch: 88   trainLoss: 1.2292e-03   valLoss:2.1285e-03  time: 7.38e+00
epoch: 89   trainLoss: 1.5395e-03   valLoss:2.5823e-03  time: 7.41e+00
epoch: 90   trainLoss: 1.5879e-03   valLoss:1.9070e-03  time: 7.36e+00
epoch: 91   trainLoss: 7.0114e-04   valLoss:1.9474e-03  time: 7.34e+00
epoch: 92   trainLoss: 8.5399e-04   valLoss:3.3999e-03  time: 7.24e+00
epoch: 93   trainLoss: 4.7510e-04   valLoss:4.2632e-03  time: 7.27e+00
epoch:

epoch: 95   trainLoss: 4.4051e-03   valLoss:8.5405e-02  time: 1.66e-01
epoch: 96   trainLoss: 2.8846e-03   valLoss:9.0409e-02  time: 1.67e-01
epoch: 97   trainLoss: 3.0451e-03   valLoss:7.9798e-02  time: 1.65e-01
epoch: 98   trainLoss: 2.3297e-03   valLoss:7.3670e-02  time: 1.65e-01
epoch: 99   trainLoss: 2.1411e-03   valLoss:7.5274e-02  time: 1.67e-01
loading checkpoint 98
loading restart file
epoch: 0   trainLoss: 1.2497e+00   valLoss:1.0558e+02  time: 1.74e-01
epoch: 1   trainLoss: 3.7625e-01   valLoss:6.8793e+00  time: 1.65e-01
epoch: 2   trainLoss: 1.8915e-01   valLoss:2.7728e+00  time: 1.66e-01
epoch: 3   trainLoss: 1.3331e-01   valLoss:2.2430e+00  time: 1.66e-01
epoch: 4   trainLoss: 1.1167e-01   valLoss:1.4621e+00  time: 1.67e-01
epoch: 5   trainLoss: 9.6433e-02   valLoss:7.7700e-01  time: 1.67e-01
epoch: 6   trainLoss: 9.3786e-02   valLoss:5.9719e-01  time: 1.73e-01
epoch: 7   trainLoss: 8.8909e-02   valLoss:5.9221e-01  time: 1.61e-01
epoch: 8   trainLoss: 8.7287e-02   valLoss

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,1.612660e-06,0.001015,0.061899,0.486692,0.536522,-3.974415,-29.030809,Fresh,Train,180
1,1.648987e-06,0.001022,0.062413,0.468563,0.506576,-4.294050,-30.468199,Fresh,Test,180
2,1.932928e-07,0.000362,0.023980,0.956268,0.954468,0.270790,-1.486604,Transfer learning,Train,180
3,2.061693e-07,0.000368,0.024227,0.945178,0.945599,0.231130,-1.780260,Transfer learning,Test,180
4,3.131397e-09,0.000028,0.001508,0.998570,1.000000,0.997160,0.992229,Random Forest,Train,180
5,2.270368e-08,0.000073,0.003875,0.989022,1.000000,0.982041,0.960091,Random Forest,Test,180
6,8.367840e-07,0.000714,0.043766,0.713401,0.767651,-1.515137,-25.614310,Fresh,Train,445
7,8.243793e-07,0.000710,0.043701,0.691233,0.741568,-1.697320,-27.940751,Fresh,Test,445
8,1.271638e-07,0.000283,0.018216,0.959450,0.979335,0.473793,-0.536891,Transfer learning,Train,445
9,1.338067e-07,0.000288,0.018385,0.951296,0.975409,0.437894,-0.740617,Transfer learning,Test,445


In [11]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_des7_tower5_03/testResults.csv')
df

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,1.612660e-06,0.001015,0.061899,0.486692,0.536522,-3.974415,-29.030809,Fresh,Train,180
1,1.648987e-06,0.001022,0.062413,0.468563,0.506576,-4.294050,-30.468199,Fresh,Test,180
2,1.932928e-07,0.000362,0.023980,0.956268,0.954468,0.270790,-1.486604,Transfer learning,Train,180
3,2.061693e-07,0.000368,0.024227,0.945178,0.945599,0.231130,-1.780260,Transfer learning,Test,180
4,3.131397e-09,0.000028,0.001508,0.998570,1.000000,0.997160,0.992229,Random Forest,Train,180
5,2.270368e-08,0.000073,0.003875,0.989022,1.000000,0.982041,0.960091,Random Forest,Test,180
6,8.367840e-07,0.000714,0.043766,0.713401,0.767651,-1.515137,-25.614310,Fresh,Train,445
7,8.243793e-07,0.000710,0.043701,0.691233,0.741568,-1.697320,-27.940751,Fresh,Test,445
8,1.271638e-07,0.000283,0.018216,0.959450,0.979335,0.473793,-0.536891,Transfer learning,Train,445
9,1.338067e-07,0.000288,0.018385,0.951296,0.975409,0.437894,-0.740617,Transfer learning,Test,445


In [12]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [14]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', scale=alt.Scale(), axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Transfer learning - tower')

alt.Chart(...)

In [65]:
trialDir = 'results/transferLrn_des7_tower5_03/00018/'
gcnHist = pd.read_csv(trialDir+'gcn/trainlog.csv', header=None, names=['epoch', 'train', 'val', 'time'])
gcnHist['trial'] = 'gcn'
ptrGcnHist = pd.read_csv(trialDir+'/ptrGcn/trainlog.csv', header=None, names=['epoch', 'train', 'val', 'time'])
ptrGcnHist['trial'] = 'ptrGcn'
df = pd.concat([gcnHist, ptrGcnHist], axis=0)

alt.Chart(df).transform_fold(
            ['train', 'val'],
            as_=['set', 'value']
            ).mark_line().encode(
                alt.X('epoch:Q', axis=alt.Axis(grid=False),),
                alt.Y('value:Q', axis=alt.Axis(title='loss', grid=False), scale=alt.Scale(domain=[0,1], clamp=True)),
                color=alt.Color('trial:N'),
                opacity=alt.Opacity('set:N'),
                tooltip=['epoch:Q', 'value:Q']
            ).properties(width=400, height=200)

alt.Chart(...)